In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.python.keras.engine.base_preprocessing_layer import PreprocessingLayer
import requests
import urllib.parse
import math
from sklearn.preprocessing import MinMaxScaler 
import matplotlib.pyplot as plt
import cupy as cp
import seaborn
from numba import cuda
import dataset
from importlib import reload
import os.path

reload(dataset)
print(tf.config.list_physical_devices('GPU'))
tf.compat.v1.enable_eager_execution()

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)  # set this TensorFlow session as the default session for Keras
keras.backend.set_epsilon(1)

[]
Device mapping: no known devices.



In [2]:
def savedata(xtrain, ytrain, xtest, ytest, name="data"):
    np.savez_compressed(os.path.join("saveddata",name), timetrain=xtrain[0], idxtrain=xtrain[1], typetrain=xtrain[2], totidxtrain=xtrain[3], ytrain=ytrain,
                       timetest=xtest[0], idxtest=xtest[1], typetest=xtest[2], totidxtest=xtest[3], ytest=ytest)
def loaddata(name="data"):
    with np.load(os.path.join("saveddata",(name if name.endswith(".npz") else name+".npz"))) as data:
        xtrain = [data["timetrain"], data["idxtrain"], data["typetrain"], data["totidxtrain"]]
        ytrain = data["ytrain"]
        xtest = [data["timetest"], data["idxtest"], data["typetest"], data["totidxtest"]]
        ytest = data["ytest"]
    return xtrain, ytrain, xtest, ytest

In [18]:
lookback=60
xtrain, ytrain, xtest, ytest = dataset.getdata(lookback=lookback, items = [
    ("★ M9 Bayonet | Fade (Factory New)", "Knife"),
    ("StatTrak™ AK-47 | Ice Coaled (Factory New)", "Gun"),
    ("Recoil Case", "Case"),
    ("CS:GO Weapon Case", "Case"),
    ("Sticker | s1mple (Gold) | Katowice 2019", "Sticker"),
    ("Sealed Graffiti | NaCl (Shark White)", "Graffiti"),
    ("Canals Pin", "Pin"),
    ("StatTrak™ Music Kit | Hundredth, FREE", "Music"),
    ("Stockholm 2021 Challengers Patch Pack", "Patch"),
    ("Number K | The Professionals", "Agent"),
    ("★ Broken Fang Gloves | Needle Point (Field-Tested)", "Gloves"),
    ("AWP | Asiimov (Well-Worn)", "Gun"),
    # ("AWP | Asiimov (Factory New)", "Gun"),
    # ("★ Broken Fang Gloves | Needle Point (Well-Worn)", "Gloves"),
])
savedata(xtrain, ytrain, xtest, ytest)

★ M9 Bayonet | Fade (Factory New)
StatTrak™ AK-47 | Ice Coaled (Factory New)
Recoil Case
CS:GO Weapon Case
Sticker | s1mple (Gold) | Katowice 2019
Sealed Graffiti | NaCl (Shark White)
Canals Pin
StatTrak™ Music Kit | Hundredth, FREE
Stockholm 2021 Challengers Patch Pack
Number K | The Professionals
★ Broken Fang Gloves | Needle Point (Field-Tested)
AWP | Asiimov (Well-Worn)
timeinputstrain.shape=(14714, 60, 1)
idxinputstrain.shape=(14714,)
typeinputstrain.shape=(14714,)
totidxinputstrain.shape=(14714,)
ytrain.shape=(14714, 1)
timeinputstest.shape=(3679, 60, 1)
idxinputstest.shape=(3679,)
typeinputstest.shape=(3679,)
totidxinputstest.shape=(3679,)
ytest.shape=(3679, 1)


In [10]:
%%time
xtrain, ytrain, xtest, ytest = loaddata()
lookback = len(xtrain[0][0])

CPU times: total: 46.9 ms
Wall time: 38.8 ms


In [11]:
rnnlayer = layers.GRU
def getmodel():
    """m = keras.Sequential([
        ,
        layers.Dense(80),
        layers.GRU(80, return_sequences=False, input_shape=(lookback, 1)),
        layers.Dense(25),
        layers.Dense(1),
    ])"""
    
    '''vocab = ["Gloves", "Agent", "Patch", "Music", "Pin", "Graffiti", "Sticker", "Case", "Knife", "Gun"]
    timeinput = keras.Input(shape=(lookback, 1))
    idxinput = keras.Input(shape=(1), dtype=tf.float32)
    typeinput = keras.Input(shape=(1), dtype=tf.string)
    totalidxinput = keras.Input(shape=(1), dtype=tf.float32)
    
    typeinput2 = layers.StringLookup(vocabulary=vocab, output_mode="multi_hot")(typeinput)
    typeinput2 = layers.Reshape((typeinput2.shape[1], 1))(typeinput2)
    
    idxinput2 = layers.Reshape((idxinput.shape[1],1))(idxinput)
    totalidxinput2 = layers.Reshape((idxinput.shape[1],1))(totalidxinput)
    
    
    x = layers.Concatenate(axis=1)([timeinput, idxinput2, typeinput2, totalidxinput2])
    x = layers.Reshape((1, x.shape[1]))(x)
    
    x = layers.Dense(200)(x)
    x = rnnlayer(200, return_sequences=True)(x)
    x = layers.Dense(120)(x)
    x = rnnlayer(120, return_sequences=True)(x)
    x = layers.Dense(80)(x)
    x = rnnlayer(80, return_sequences=True)(x)
    x = layers.Dense(40)(x)
    x = rnnlayer(40, return_sequences=False)(x)
    # x = layers.Dense(25)(x)
    for i in range(5):
        x = layers.Dense(25)(x)
    x = layers.Dense(1)(x)
    x = layers.Reshape((1,))(x)
    x = layers.ReLU()(x)
    
    m = keras.Model(inputs=[timeinput, idxinput, typeinput, totalidxinput], outputs=x)
    
    m.build((None, lookback, 1))
    m.summary()
    return m'''
    
    timeinput = keras.Input(shape=(lookback, 1))
    idxinput = keras.Input(shape=(1), dtype=tf.float32)
    typeinput = keras.Input(shape=(1), dtype=tf.string)
    totalidxinput = keras.Input(shape=(1), dtype=tf.float32)

    vocab = ["Case"]

    typeinput2 = layers.StringLookup(vocabulary=vocab, output_mode="multi_hot")(typeinput)
    typeinput2 = layers.Reshape((typeinput2.shape[1], 1))(typeinput2)

    idxinput2 = layers.Reshape((idxinput.shape[1], 1))(idxinput)
    totalidxinput2 = layers.Reshape((idxinput.shape[1], 1))(totalidxinput)
    timeinput2 = layers.Normalization()(timeinput)

    for i in range(5):
        timeinput2 = rnnlayer(80, return_sequences=True)(timeinput2)
    timeinput2 = rnnlayer(80, return_sequences=False)(timeinput2)
    timeinput2 = layers.Reshape((timeinput2.shape[1], 1))(timeinput2)

    x = layers.Concatenate(axis=1)([timeinput2, idxinput2, typeinput2, totalidxinput2])
    x = layers.Reshape((1, x.shape[1]))(x)

    for i in range(20):
        x = layers.Dense(20)(x)

    x = layers.Dense(1)(x)
    m = keras.Model(inputs=[timeinput, idxinput, typeinput, totalidxinput], outputs=x)

    m.build((None, lookback, 1))
    m.summary()
    return m
model=getmodel()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 60, 1)]      0           []                               
                                                                                                  
 normalization (Normalization)  (None, 60, 1)        3           ['input_1[0][0]']                
                                                                                                  
 gru (GRU)                      (None, 60, 80)       19920       ['normalization[0][0]']          
                                                                                                  
 gru_1 (GRU)                    (None, 60, 80)       38880       ['gru[0][0]']                    
                                                                                              

In [12]:
%%time
model.compile(optimizer="adam",loss="mean_absolute_percentage_error", metrics=["accuracy", keras.metrics.RootMeanSquaredError(), keras.metrics.MeanAbsolutePercentageError()], callbacks=[keras.callbacks.EarlyStopping(monitor='loss', patience=3)])
history=model.fit(xtrain, ytrain, batch_size=32, epochs=100, verbose=1)

Epoch 1/100
339/460 [=====================>........] - ETA: 13s - loss: 156.6864 - accuracy: 9.2183e-05 - root_mean_squared_error: 252.3463 - mean_absolute_percentage_error: 156.6864


KeyboardInterrupt



In [7]:
model.save("model.hdf5")

In [8]:
model = models.load_model("model.hdf5")

In [9]:
pred = model.predict(xtrain)
print(pred.shape)
print(ytrain.shape)
print(type(pred))
# pred = scaler.inverse_transform(pred)

print("Training MAPE",np.mean(np.abs((ytrain - pred) / ytrain)) * 100)
print("Average testing distance percentage:", np.mean(np.divide(np.absolute(pred-ytrain),ytrain))*100)
print(ytrain)
print(pred)

460/460 [==============================] - 2s 2ms/step
(14714, 1)
(14714, 1)
<class 'numpy.ndarray'>
Training MAPE 100.0
Average testing distance percentage: 100.0
[[ 36.12]
 [ 33.81]
 [325.39]
 ...
 [  2.71]
 [  6.72]
 [  3.82]]
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]


In [10]:
pred = model.predict(xtest)
print("Testing MAPE",np.mean(np.abs((ytest - pred) / ytest)) * 100)
print("Average testing distance percentage:", np.mean(np.divide(np.absolute(pred-ytest),ytest))*100)
print(ytest)
print(pred)

115/115 [==============================] - 0s 2ms/step
Testing MAPE 100.0
Average testing distance percentage: 100.0
[[3.0000e-02]
 [4.0285e+02]
 [9.9500e+00]
 ...
 [1.2884e+03]
 [6.8190e+01]
 [6.7950e+01]]
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]


In [11]:
model.evaluate(xtrain, ytrain)

460/460 [==============================] - 2s 2ms/step - loss: 88.1373 - accuracy: 0.0000e+00 - root_mean_squared_error: 250.0299 - mean_absolute_percentage_error: 88.1373


[88.13728332519531, 0.0, 250.0298614501953, 88.13728332519531]

In [12]:
model.evaluate(xtest, ytest)

115/115 [==============================] - 0s 3ms/step - loss: 87.6233 - accuracy: 0.0000e+00 - root_mean_squared_error: 257.5083 - mean_absolute_percentage_error: 87.6233


[87.6232681274414, 0.0, 257.5083312988281, 87.6232681274414]

In [13]:
model.predict([np.zeros((1,60, 1)), np.asarray([42069]), np.asarray(["Knife"]), np.asarray([100100])])

1/1 [==============================] - 0s 129ms/step


array([[0.]], dtype=float32)

In [14]:
tf.keras.backend.clear_session()

In [15]:
device = cuda.get_current_device()
device.reset()